In [1]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import hashlib
import time
from jupyter_util import *
from importlib import reload

In [2]:
# Connect agents
agents = [8150,8151]
connection = [1,2]
inv = [0,1]
get_index = lambda x: 1 if x == 0 else 0

for i in range(len(connection)):
    inv[i] = post(agents[i], "/connections/create-invitation?alias=agent_connection&auto_accept=true&multi_use=true")
    # Create then register DIDS 
    resp = post(agents[i],  "/wallet/did/create")
    dids = requests.post("http://localhost:9000/register", json={
        "did": resp['result']['did'], 
        "verkey":resp['result']['verkey'], 
        "alias": "BOB"+str(agents[i]), 
        "role": "ENDORSER"
    })
    resp = post(agents[i], "/wallet/did/public?did=" + resp['result']['did'])

for i in range(len(connection)):
    connection[i] = post(agents[i], "/connections/receive-invitation?auto_accept=true", {
        "label": "Bob",
        "recipientKeys": inv[get_index(i)]['invitation']['recipientKeys'],
        "serviceEndpoint": inv[get_index(i)]['invitation']['serviceEndpoint'],
    })['connection_id']

# Services

In [3]:
set_pds_config_agent1(
    "own_your_data",
    {
        "client_id": "gVQJTSabNK8DxNzu3PeGWQwBtXSb2Iv7FgYlbzBEbfg",
        "client_secret": "9yBmtiyO2YwhtHA9btSrYensVZZ9DLNY1Vq5D-EjxwQ",
        "api_url": "https://data-vault.eu",
    },
)

set_pds_config_agent2(
    "own_your_data",
    {
        "client_id": "nM8p2yYuxqGWvbJ50t8ODifjyFZfi-yOm2HoE6AZaW0",
        "client_secret": "X6WBjmMylTQ3BdbBRKkBir3uVz79RgADetw5tl7Tgyo",
        "api_url": "https://data-vault.eu",
    },
)

# set_pds_config_agent2(
#     "own_your_data",
#     { "scope": "admin", "client_id": "50c67447e28eeb5ac56bd8bc3fbfbc5916f817603eb3f117f01e84d52e36afb7", "client_secret": "6f20318f0a451b24a9c4aaa00214626d6e0f6b15d18957acd11a085c4d7b2dbd", "api_url": "https://dip-officer.data-container.net" }
# )


{"success": "True", "status": {"own_your_data": {"connected": true}}}
{"success": "True", "status": {"own_your_data": {"connected": true}}}


In [4]:
def __pds_test_issue_credential():
    issue_cred_request = post(
        8150,
        "/issue-credential/request",
        {
            "credential_values": {
                "type": "Person",
                "address": {
                    "type": "PostalAddress",
                    "streetAddress": "123 Main St.",
                    "addressLocality": "Blacksburg",
                    "addressRegion": "VA",
                    "postalCode": "24060",
                    "addressCountry": "US",
                },
            },
            "connection_id": connection[0],
        },
    )
    time.sleep(0.1)
    cred_exchange = get(
        8151,
        f"/issue-credential/exchange/record?thread_id={issue_cred_request['thread_id']}",
    )
    cred_exchange = cred_exchange["result"]
    id = cred_exchange[0]["credential_exchange_id"]
    issue_cred = post(8151, "/issue-credential/issue?credential_exchange_id=" + id)
time_this(100, "time_rfc36_issue_credential.json", __pds_test_issue_credential)

0.43465445000038017
0.4115078230006475
0.39992815799996606
0.4203190949992859
0.42281911600002786
0.3874226190000627
0.4092392530001234
0.44287355300002673
0.39723517799939145
0.408308850001049
0.41102637000039977
0.4371841089996451
0.38634619600088627
0.41569086100025743
0.4330844470005104
0.42508662000000186
0.40083523300017987
0.40001582500008226
0.3937298849996296
0.4200462160006282
0.4413409459994
0.47355982799854246
0.42151819100035937
0.4140625510008249
0.3994318560016836
0.47574432600049477
0.4272064519991545
0.3828578850007034
0.3780579489994125
0.5035336820001248
0.4359233499999391
0.46059521099959966
0.4144112319991109
0.43323486300141667
0.3987252309998439
0.420976626999618
0.3946601749994443
0.3874273510009516
0.4321256410003116
0.43441363099918817
0.4644732380002097
0.4611485130008077
0.4279622499998368
0.4588022930001898
0.45697627600020496
0.40146960400124954
0.4285645440013468
0.4171633429996291
0.4210239259991795
0.3857031150000694
0.4329613390000304
0.438269827000112

In [5]:
table = get_time_table()
for i in range(100):
    table['total_time'][i] -= 0.1

In [6]:
with open("time_rfc36_issue_credential_final.json", 'w') as outfile:
    json.dump(table, outfile, sort_keys=True, indent=4)
    print("DONE!")

DONE!


In [72]:
import random
add_consent = post(8151, "/verifiable-services/consents", {
    "label": "test_" + str(random.randint(1, 100000000)),
    "oca_schema_namespace": "consent",
    "oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s",
    "oca_data": {
        "expiration": "7200",
        "limitation": "7200",
        "dictatedBy": "test",
        "validityTTL": "7200",
    }
})
get_consents_provider = get(8151, "/verifiable-services/consents")
get_consents_user = get(8150, "/verifiable-services/consents")

save_data_to_pds = post(8151, "/pds/save", {"payload": json.dumps({
        "associatedReportID": "Text12",
        "uniqueNumericIDHash": "Text",
        "disease": "Text",
        "vaccineDescription": "Text",
        "atcCode": "Text",
        "certificateIssuer": "Text",
        "certificateNumber": "Text",
        "certificateValidFrom": "Date",
        "certificateValidTo": "Date",
        "formVersion": "Text",
    }), "oca_schema_dri": "dip.data.tda.oca_chunks.predefined.9GdWoQwth9299oYj8HfdgRZjtSxW9sTVyy1RnfhQ35yA"
})

In [73]:
add = post(8151, '/verifiable-services/add', {
  "label": "consent_test",
  "service_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "12345"
  },
  "certificate_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "9GdWoQwth9299oYj8HfdgRZjtSxW9sTVyy1RnfhQ35yA"
  },
  "consent_id": add_consent['consent_id']
})

In [17]:
with open("time_services.json", 'w') as outfile:
    json.dump(time_serv, outfile, sort_keys=True, indent=4)
    print("DONE!")

DONE!


In [11]:
time_serv = {**time1, **time2}

In [74]:
service_list = get(8150, '/verifiable-services/DEBUGrequest/' + connection[0])
webhook_service_list = get(8150, '/verifiable-services/request-service-list/' + connection[0])

In [20]:
for i in range(100):
    time_serv['total_time'][i] -= time_serv["8150/verifiable-services/DEBUGrequest/a28cbd22-a91b-4c69-9178-1aa2e1f99db3"][i]

IndexError: list index out of range

In [75]:
service = service_list['result']['services'][-1]

In [24]:
len(time_serv["8150/verifiable-services/DEBUGrequest/a28cbd22-a91b-4c69-9178-1aa2e1f99db3"])

100

In [76]:
# service = None
# services = service_list['result']['services']
# for i in services:
#     if i['service_id'] == add['service_id']:
#         service = i
# assert service == add['service_id']

In [77]:
service = service_list['result']['services'][0]
service.pop("policy_validation", None)
apply = post(8150, '/verifiable-services/apply', {
  "connection_id": connection[0],
  "user_data": '{"memes": "films"}',
  "service": service,
})
print(apply)

{'success': True, 'exchange_id': 'bf5d2461-985d-490b-b385-c839c94cd34e'}


In [78]:
time.sleep(1)
issue_self = requests.post('http://localhost:8151/verifiable-services/get-issue', json={
    "exchange_id": apply['exchange_id'],
})
issue_self = json.loads(issue_self.text)
issue_self = issue_self['result']

In [79]:
process_application = post(8151, '/verifiable-services/process-application', {
    "issue_id": issue_self[0]['issue_id'],
    "decision": "accept",
    "data": {
        "test": "certificate"
    }
})


In [80]:
issue_self = post(8151, '/verifiable-services/get-issue', {
    "exchange_id": apply['exchange_id'],
})['result'][0]
issue_self['consent_schema'].pop("usage_policy")
print(issue_self['state'])

accepted


In [112]:
issue_self = post(8150, '/verifiable-services/get-issue', {
    "exchange_id": apply['exchange_id'],
})['result']
print(issue_self[0]["state"])
result = get(8150, "/verifiable-services/report/" + issue_self[0]['report_data_dri'])
print(result)

credential_received
{}


In [115]:

print(report)

{'test': 'certificate'}


In [118]:
cred = get(8151, "/pds/" + issue_self[0]["credential_id"])
user_data = get(8151, "/pds/" + issue_self[0]["service_user_data_dri"])
consent = get(8151, "/pds/" + issue_self[0]["user_consent_credential_dri"])
report = get(8151, "/pds/" + issue_self[0]["report_data_dri"])
cred_content = get(8151, "/pds/" + cred['payload']['credentialSubject']['oca_data_dri'])
report = get(8151, "/verifiable-services/report/" + cred_content['payload']['associatedReportID'])

In [120]:
cred = get(8150, "/pds/" + issue_self[0]["credential_id"])
user_data = get(8150, "/pds/" + issue_self[0]["service_user_data_dri"])
consent = get(8150, "/pds/" + issue_self[0]["user_consent_credential_dri"])
report = get(8150, "/pds/" + issue_self[0]["report_data_dri"])
cred_content = get(8150, "/pds/" + cred['payload']['credentialSubject']['oca_data_dri'])
report = get(8150, "/verifiable-services/report/" + cred_content['payload']['associatedReportID'])

In [7]:
credentials = requests.get('http://localhost:8150/credentials')
credentials = json.loads(credentials.text) 
pp.pprint(credentials)

{'result': [{'content': {'context': ['https://www.w3.org/2018/credentials/v1',
                                     'https://www.schema.org'],
                         'credentialSubject': {'oca_data_dri': 'zQmT5xhkWavzHajLN6pGwk7WH9eVxBHjCFDa8C1rcEHheWu',
                                               'oca_schema_dri': '9GdWoQwth9299oYj8HfdgRZjtSxW9sTVyy1RnfhQ35yA',
                                               'oca_schema_namespace': 'string',
                                               'service_consent_match_id': 'f5ae82f7-1536-4aa6-aaaa-702bf6f58e5b',
                                               'subject_id': 'JcsaprS8G2ACN7jrQYdQdb'},
                         'issuanceDate': '2021-05-07 10:57:39.927342Z',
                         'issuer': 'KPrFzPBnahcVcjR4pGdp8f',
                         'proof': {'created': '2021-05-07 10:57:39.932962Z',
                                   'jws': 'Mpw2kVipTzPz3pm6A-PuGnKKzgXDOF3w0doQU7QwqvGWA5GrciHYpm6AQqR7rb27T63aeNTMiaKCkeG3T_EQAA',
    

In [8]:
credential = credentials['result'][-2]['content']
print(credential['credentialSubject']['oca_schema_dri'])

9GdWoQwth9299oYj8HfdgRZjtSxW9sTVyy1RnfhQ35yA


In [ ]:
exchange_record = get(
    8150,
    f"/present-proof/exchange/record?state=request_received",
)
exchange_record

In [9]:
def test_present_proof():
    request_proof = post(
        8151,
        "/present-proof/request",
        {
            "connection_id": connection[1],
            "schema_base_dri": credential['credentialSubject']['oca_schema_dri'],
            "requested_attributes": ["address"],
            "issuer_did": "123145",
        },
    )
    exchange_id_agent2 = request_proof["exchange_id"]
    exchange_record = {"result": []}
    while exchange_record.get("result") == []:
        exchange_record = get(
            8150,
            f"/present-proof/exchange/record?state=request_received",
        )

    # # FETCH CREDENTIALS (1)
    creds = get(8150, "/credentials")

    # # PRESENT PROOF (1)
    present = post(
        8150,
        "/present-proof/present",
        {
            "credential_id": creds["result"][0]["dri"],
            "exchange_record_id": exchange_record["result"][0][
                "presentation_exchange_id"
            ],
        },
    )

    exchange_record = get(
        8151,
        f"/present-proof/exchange/record?connection_id={connection[1]}&thread_id={request_proof['thread_id']}",
    )

    time.sleep(1)
    exchange_record = get(
        8151,
        f"/present-proof/exchange/record?state=presentation_received",
    )

    issue_cred = post(
        8151,
        f"/present-proof/acknowledge?status=true&exchange_record_id={exchange_record['result'][0]['presentation_exchange_id']}",
    )

time_this(100, "time_rfc37_present_proof.json", test_present_proof)

6.267038410000168
5.910699357998965
5.853009553000447
5.457997118001003
6.0321916960001545
6.096818551000979
6.3815862579995155
6.237135516001217
7.812879267999961
6.053860866999457
5.359344768001392
6.168187704000957
6.113185006001004
5.948464483999487
6.457784996000555
6.170168908000051
6.077126793999923
6.286335168999358
6.151284522000424
6.282844340999873
5.5069983980010875
5.6025038579991815
5.8735632049993
6.173445307000293
6.176317673000085
6.184752238999863
6.1357174389995635
6.140214659999401
6.227509248001297
5.6788815739982965
6.219993054000952
6.5400668530000985
6.334946521999882
6.179237687998466
6.361409646999164
5.708253812999828
6.189637187999324
6.396028057000876
6.2200016120004875
6.086418409000544
6.075666964999982
5.976329324999824
6.233706877001168
5.645194052998704
6.396998109999913
6.421787438999672
6.097858391000045
6.0621331419988564
6.27439198600041
6.056526735999796
6.3712052890005
5.295703502000833
5.587422814000092
5.613803575999555
5.570185341999604
5.6924

In [11]:
import copy
copy_table = copy.deepcopy(get_time_table())

In [13]:
table = get_time_table()
for i in range(100):
    table['total_time'][i] -= 1

In [14]:
len(get_time_table()['total_time'])

100

In [15]:
get_time_table()

{'8150/connections/create-invitation': [0.032180772999709006],
 '8150/wallet/did/create': [0.008286792999570025],
 '8150/wallet/did/public': [3.0119548959992244],
 '8151/connections/create-invitation': [0.02712740000060876],
 '8151/wallet/did/create': [0.007946388999698684],
 '8151/wallet/did/public': [3.012620475999938],
 '8150/connections/receive-invitation': [0.04850051399989752],
 '8151/connections/receive-invitation': [0.06359022700053174],
 'total_time': [5.267038410000168,
  4.910699357998965,
  4.853009553000447,
  4.457997118001003,
  5.0321916960001545,
  5.096818551000979,
  5.3815862579995155,
  5.237135516001217,
  6.812879267999961,
  5.053860866999457,
  4.359344768001392,
  5.168187704000957,
  5.113185006001004,
  4.948464483999487,
  5.457784996000555,
  5.170168908000051,
  5.077126793999923,
  5.286335168999358,
  5.151284522000424,
  5.282844340999873,
  4.5069983980010875,
  4.6025038579991815,
  4.8735632049993,
  5.173445307000293,
  5.176317673000085,
  5.18475

In [16]:
with open('time_rfc37_present_proof_final.json', 'w') as outfile:
    json.dump(table, outfile, sort_keys=True, indent=4)